In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df.head(10)

In [ ]:
df_train = df[['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]
df_train.describe()


In [ ]:
import seaborn as sns

sns.heatmap(df_train.corr(), vmin=-1, vmax=1,  cmap='YlGnBu', annot=True)
plt.show()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
#stratified shuffle split picks features with similar distribution between train and test (cross validation behaviour)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for train_indices, test_indices in split.split(df, df[['Survived', 'Sex', 'Pclass']]):
    strat_df_train = df.loc[train_indices]
    strat_df_test = df.loc[test_indices]

In [ ]:
strat_df_train.info()

In [ ]:
#replace NaN values with mean

age_mean = df['Age'].mean()

strat_df_train['Age'].fillna(value=age_mean, inplace=True)
strat_df_test['Age'].fillna(value=age_mean, inplace=True)

strat_df_train.info()

In [ ]:
strat_df_train = strat_df_train.drop(['Name','Cabin', 'Ticket'], axis=1)
strat_df_test = strat_df_test.drop(['Name','Cabin','Ticket'], axis=1)


In [ ]:
strat_df_train = pd.get_dummies(strat_df_train)
strat_df_test = pd.get_dummies(strat_df_test)
print(strat_df_train)

In [ ]:
strat_df_train.info()

In [ ]:
from sklearn.preprocessing import StandardScaler

X = strat_df_train.drop(['Survived'], axis=1)
y = strat_df_train['Survived']

scaler = StandardScaler()
X_data = scaler.fit_transform(X)
y_data = y.to_numpy()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model = RandomForestClassifier()

param_grid = [
    {"n_estimators": [100,200,500], "max_depth": [1, 5, 10], "min_samples_split" : [2,4,6]}
]

search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', return_train_score=True)
search.fit(X_data, y_data)

In [ ]:
final_model = search.best_estimator_
final_model

In [ ]:
X_test = strat_df_test.drop(['Survived'], axis=1)
y_test = strat_df_test['Survived']

scaler = StandardScaler()
X_data_test = scaler.fit_transform(X_test)
y_data_test = y_test.to_numpy()

In [ ]:
final_model.score(X_data_test, y_data_test)

In [ ]:
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
df.info()
df_test.info()

In [ ]:

df['Age'].fillna(value=age_mean, inplace=True)

fare_mean_test = df_test['Fare'].mean()
age_mean_test = df_test['Age'].mean()
df_test['Fare'].fillna(value=fare_mean_test, inplace=True)
df_test['Age'].fillna(value=age_mean_test, inplace=True)
df_test.info()

In [ ]:
df_train = df.drop(['Name','Cabin','Ticket'], axis=1)
df_test = df_test.drop(['Name','Cabin','Ticket'], axis=1)

df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)
df_train.info()
df_test.info()

In [ ]:
X_final = df_train.drop(['Survived'], axis=1)
y_final = df_train['Survived']

X_data_final = scaler.fit_transform(X_final)
y_data_final = y_final.to_numpy()

model = RandomForestClassifier()
search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy', return_train_score=True)
search.fit(X_data_final, y_data_final)

In [ ]:
final_data_model = search.best_estimator_
final_data_model

In [ ]:
final_data_model.score(X_data_final, y_data_final)

In [ ]:

final_df_test = scaler.fit_transform(df_test)

predictions = final_data_model.predict(final_df_test)

In [ ]:
output_df = pd.DataFrame(df_test['PassengerId'])
output_df['Survived']= predictions
output_df.to_csv('submission.csv', index=False)

print('Your csv was saved')

In [ ]:
output_df